|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating layers<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Laminar profile of RSA and cateogry selectivity<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from transformers import AutoModelForCausalLM, GPT2Tokenizer

# vector matplotlib plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1: Hook all layers and get activations

In [ ]:
# load GPT2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2-large')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

model.eval()

In [ ]:
# Define a hook function to store QVK vectors
activations = {}

def implant_hook(layer_number):
  def hook(module, input, output):

    # get and separate the QVK
    q,k,v = .split(

    # numpyify and store
    activations[f'attn_{layer_number}_q'] =
    activations[f'attn_{layer_number = # k
    activations[ = # v
  return hook


# surgery ;)
for layeri in range(model.config.n_layer):
  model.......register_forward_hook(implant_hook(layeri))

In [ ]:
# list of words for RSA
words = [ 'galaxy','asteroid','comet','cosmos','space','sun','planet','moon','star','orbit',
          'ceiling','sofa','couch','carpet','door','window','lamp','chair','table','rug','bed','floor','wall',
          'pear','grape','banana','cherry','peach','apple','seed','jelly','orange','lime','fruit'
        ]

# first create a group mask based on word order
group = np.array([ [1,1,1,1,1,1,1,1,1,1,
                    2,2,2,2,2,2,2,2,2,2,2,2,2,
                    3,3,3,3,3,3,3,3,3,3,3
                    ] ])
groupmask = np.triu( group.T@group ,1)

In [ ]:
batch = torch.zeros((len(words),5),dtype=torch.long)

for i,w in enumerate(words):
  batch[i,:] = torch.tensor(tokenizer.encode(f'The next word is

# push through the model
with torch.no_grad(): model(batch)

In [ ]:
activations.keys(),activations['attn_5_q'].shape

# Exercise 2: Cosine similarities in all layers

In [ ]:
# initializations
selectIndices = np.zeros((3,model.config.n_layer))
RSA = np.zeros((3,model.config.n_layer))


for layeri in range(model.config.n_layer):


  ### cosine similarity matrices

  # for Q
  actsQ = activations[f'attn_{layeri}_q'] # activations for the target token
  actsQ /= np.linalg.norm( # normalize
  cs_actsQ = # QQ'

  # for K

  cs_actsK =

  # for V

  cs_actsV =



  ### selectivity indices
  selectnum = ( cs_actsQ[groupmask==1].mean() + cs_actsQ[groupmask==4].mean() + cs_actsQ[groupmask==9].mean() ) / # scale to average
  selectden = ( cs_actsQ[].mean() + cs_actsQ[].mean() + cs_actsQ[].mean() ) / # the off-diagonal blocks
  selectIndices[0,layeri] = selectnum/selectden

  selectnum = # repeat for K
  selectden =
  selectIndices[1,layeri] = selectnum/selectden

  selectnum = # repeat for V
  selectden =
  selectIndices[2,layeri] = selectnum/selectden


  ### RSA
  # extract the upper-triangular elements
  unique_Q = cs_actsQ[
  unique_K = cs_actsK[
  unique_V = cs_actsV[


  # Pearson correlations to get RSA
  RSA[0,layeri] = np.corrcoef(unique_Q,unique_K) # Q with K
  RSA[1,layeri] = np.corrcoef # Q with V
  RSA[2,layeri] =  # K with V


# Exercise 3: Laminar profiles of selectivity and RSA

In [ ]:
_,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(selectIndices[0],'kv',markersize=8,markerfacecolor=[.9,.7,.7],label='Q')
axs[0].plot(,'ko',markersize=8,markerfacecolor=[.7,.9,.7],label='K')
axs[0].plot(,'ks',markersize=8,markerfacecolor=[.7,.7,.9],label='V')
axs[0].legend()
axs[0].set(xlabel='Transformer layer',ylabel='Selectivity index')

axs[1].plot(RSA...,label='Q-K')
axs[1].plot(,label='Q-V')
axs[1].plot(,label='K-V')
axs[1].set(xlabel='Transformer layer',ylabel='RSA')
axs[1].legend()

plt.tight_layout()
plt.show()